https://huggingface.co/ibm-nasa-geospatial

https://huggingface.co/ibm-nasa-geospatial/Prithvi-100M

In [1]:
%load_ext autoreload
%autoreload 2

The model accepts remote sensing data in a video format (B, C, T, H, W). he model can also handle static imagery which can be fed into the model with T=1. 


The model was pre-trained with NASA's HLS V2 L30 product (30m granularity) from the contiguous United States. The bands that were used are the following:

- Blue
- Green
- Red
- Narrow NIR
- SWIR 1
- SWIR 2

Image size is 224x224. 

In [4]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import yaml
from prithvi.Prithvi import MaskedAutoencoderViT

NO_DATA = -9999
NO_DATA_FLOAT = 0.0001
PERCENTILES = (0.1, 99.9)

In [5]:
# load weights
weights_path = "./prithvi/Prithvi_100M.pt"
checkpoint = torch.load(weights_path, map_location="cpu")

# read model config
model_cfg_path = "./prithvi/Prithvi_100M_config.yaml"
with open(model_cfg_path) as f:
    model_config = yaml.safe_load(f)

model_args, train_args = model_config["model_args"], model_config["train_params"]

# let us use only 1 frame for now (the model was trained on 3 frames)
model_args["num_frames"] = 1

# instantiate model
model = MaskedAutoencoderViT(**model_args)
model.eval()

# load weights into model
# strict=false since we are loading with only 1 frame, but the warning is expected
del checkpoint['pos_embed']
del checkpoint['decoder_pos_embed']
_ = model.load_state_dict(checkpoint, strict=False)

In [9]:
output = model(torch.rand(1, 6, 1, 224, 224))
output

(tensor(0.0978, grad_fn=<DivBackward0>),
 tensor([[[0.3786, 0.3884, 0.4116,  ..., 0.4292, 0.5119, 0.5209],
          [0.4134, 0.4202, 0.4398,  ..., 0.4632, 0.4626, 0.4618],
          [0.4605, 0.4989, 0.5138,  ..., 0.3287, 0.3848, 0.3813],
          ...,
          [0.3458, 0.3760, 0.3573,  ..., 0.4014, 0.7071, 0.8298],
          [0.4271, 0.4286, 0.4310,  ..., 0.5444, 0.7183, 0.7511],
          [0.4569, 0.4543, 0.4532,  ..., 0.3873, 0.5268, 0.5622]]],
        grad_fn=<SliceBackward0>),
 tensor([[1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
          1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
          1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
          1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
          1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
          0., 1., 0

The model returns a tuple with:

- loss
- reconstructed image
- mask used

Images are normalized

In [11]:
# statistics used to normalize images before passing to the model
means = train_args["data_mean"]
stds = train_args["data_std"]

means, stds

([775.2290211032589,
  1080.992780391705,
  1228.5855250417867,
  2497.2022620507532,
  2204.2139147975554,
  1610.8324823273745],
 [1281.526139861424,
  1270.0297974547493,
  1399.4802505642526,
  1368.3446143747644,
  1291.6764008585435,
  1154.505683480695])

To finetune, you can now write a PyTorch loop as usual to train on your dataset. Simply extract the backbone from the model with some surgery and run only the model features forward, with no masking!

In general some reccomendations are:

- At least in the beggining, experiment with freezing the backbone. This will give you much faster iteration through experiments.
- Err on the side of a smaller learning rate
- With an unfrozen encoder, regularization is your friend! (Weight decay, dropout, batchnorm...)

In [14]:
features, _, _ = model.forward_encoder(torch.rand(1, 6, 1, 224, 224), mask_ratio=0)
features.shape

torch.Size([1, 197, 768])

These are the standard output of a ViT.

- Dim 1: Batch size
- Dim 2: [cls_token] + tokens representing flattened image
- Dim 3: embedding dimension

In [15]:
print(f"Encoder features have shape {features.shape}")

# drop cls token
reshaped_features = features[:, 1:, :]

# reshape
feature_img_side_length = int(np.sqrt(reshaped_features.shape[1]))
reshaped_features = reshaped_features.view(-1, feature_img_side_length, feature_img_side_length, model_args["embed_dim"])
# channels first
reshaped_features = reshaped_features.permute(0, 3, 1, 2)
print(f"Encoder features have new shape {reshaped_features.shape}")

Encoder features have shape torch.Size([1, 197, 768])
Encoder features have new shape torch.Size([1, 768, 14, 14])
